<a href="https://colab.research.google.com/github/zakmoss/OpenPose_maneuver_grader/blob/main/Openpose_data_preprocessing_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# prev

import os
import json
import numpy as np
import cv2
import os

#before running script you need the jsons and the original video. place the json folder and the video within a folder in your google drive

# 1. CREATE A FOLDER in google drive that contains the json folder file. also, put the original video in there too.
# 2. DEFINE THESE VARIABLES BEFORE

# parent folder name with a / at the end. for example, if named example, insert 'example/'. dont forget to add the sub-parent folder
input_folder_name = 'Surfing with Pose Estimation/rhGeneral/example62/'

#  folder with jsons and video.
input_jsons = '62_jsons'
input_video = '62.avi'

# input the video name without .avi at the end. for example, if input_video is surfVid.avi, input_jpg will be surfVid
input_jpg = '62'


def json_to_npy_converter(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.endswith(".json"):
            input_file_path = os.path.join(input_folder, file_name)

            with open(input_file_path, 'r') as file:
                try:
                    data = json.load(file)
                    for person in data['people']:
                        keypoints = person['pose_keypoints_2d']
                        keypoints_array = np.array(keypoints).reshape(-1, 3)
                        output_file_path = os.path.join(output_folder, file_name.replace('.json', '.npy'))
                        np.save(output_file_path, keypoints_array)
                        print(f"Saved NPY file: {output_file_path}")
                except Exception as e:
                    print(f"Error processing file {input_file_path}: {e}")

# Replace the example paths with your actual paths
input_folder = '/content/drive/My Drive/' + input_folder_name + input_jsons
output_folder =  '/content/drive/My Drive/' + input_folder_name + '/output1'
json_to_npy_converter(input_folder, output_folder)


# video path extraction

def extract_first_frame(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file.")
        return

    success, frame = cap.read()
    if success:
        output_filename = os.path.splitext(os.path.basename(video_path))[0] + "frame.jpg"
        output_path = os.path.join(os.path.dirname(video_path), output_filename)

        cv2.imwrite(output_path, frame)
        print(f"Saved first frame to {output_path}")
    else:
        print("Failed to extract the first frame.")

    cap.release()
#replace example path with actual path
video_path = '/content/drive/My Drive/' + input_folder_name + input_video
extract_first_frame(video_path)

#finds frame center

from PIL import Image

def get_center_coordinates(image_path):
    with Image.open(image_path) as img:
        with Image.open(image_path):
            height, width = img.size
            center_x = width/2
            center_y = height/2
            return center_x,center_y




image_path = '/content/drive/My Drive/' + input_folder_name + input_jpg + 'frame.jpg'
center_coordinates = get_center_coordinates(image_path)
print(f"{center_coordinates}")

#resize pose estimates to consistent size (height = 220 pixels)

import numpy as np
import os

def rescale_pose_estimates(input_folder, output_folder, target_height=220):


    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.npy'):
            file_path = os.path.join(input_folder, file_name)
            pose_estimates = np.load(file_path)

            # Calculate the current height
            max_y = np.max(pose_estimates[:, 1])
            min_y = np.min(pose_estimates[:, 1])
            current_height = max_y - min_y


            scaling_factor = target_height / current_height if current_height > 0 else 0


            pose_estimates[:, :2] *= scaling_factor


            new_file_path = os.path.join(output_folder, file_name)
            np.save(new_file_path, pose_estimates)

#replace the example paths with actual paths
input_folder = output_folder
output_folder = '/content/drive/My Drive/' + input_folder_name + '/output2'


rescale_pose_estimates(input_folder, output_folder)

#center keypoints in frame
#use output from get_center_coordinates for "frame_center" variable (duh)
#input folder is the ouput folder from rescale_pose_estimate

import numpy as np
import os

def center_and_save_pose_estimates(input_folder, output_folder, frame_center):

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)


    for file_name in os.listdir(input_folder):
        if file_name.endswith('.npy'):
            file_path = os.path.join(input_folder, file_name)
            keypoints = np.load(file_path)


            center_of_mass = np.mean(keypoints[:, :2], axis=0)

            difference = center_of_mass-frame_center

            centered_keypoints = keypoints.copy()


            centered_keypoints[:, :2] -= difference


            output_path = os.path.join(output_folder, file_name)
            np.save(output_path, centered_keypoints)


input_folder = output_folder
output_folder = '/content/gdrive/My Drive/' + input_folder_name + '/output3'
frame_center = center_coordinates

center_and_save_pose_estimates(input_folder, output_folder, frame_center)
"subtract difference between reference point and center point from each coordinate"
#check pose heights from each frame

import numpy as np
import os

def calculate_pose_heights(directory):

    heights = {}

    for file_name in os.listdir(directory):
        if file_name.endswith('.npy'):
            file_path = os.path.join(directory, file_name)
            keypoints = np.load(file_path)

            height = np.max(keypoints[:, 1]) - np.min(keypoints[:, 1])
            heights[file_name] = height

    return heights

#replace the example paths with actual paths
directory =  output_folder


pose_heights = calculate_pose_heights(directory)
for file_name, height in pose_heights.items():
    print(f"{file_name}: {height}")

    #seperate pose estimates into arrays corresponding to keypoint ordering ^

import numpy as np
import os


def bodypart_array_maker(input_folder, output_folder):

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    keypoints_accumulator = {} # a dictionary where the key refers to the keypoint index and the value is a list containing the corresponding keypoint accross each frame
    #dictionary contains 25 indeces and each index contains a list that is 319 long
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.npy'):
            file_path = os.path.join(input_folder, file_name)
            keypoints = np.load(file_path)

            for i, keypoint in enumerate(keypoints):
                if i not in keypoints_accumulator:
                    keypoints_accumulator[i] = []   #creates list for each keypoint
                keypoints_accumulator[i].append(keypoint) #populates each list one by one

        for keypoint_index, keypoint_list in keypoints_accumulator.items():
            output_filepath = os.path.join(output_folder, f'keypoint_{keypoint_index}.npy')
            np.save(output_filepath, np.array(keypoint_list))





input_folder = output_folder
output_folder = '/content/drive/My Drive/' + input_folder_name + '/output4'
bodypart_array_maker(input_folder, output_folder)


import pandas as pd
import os
import numpy as np
import scipy as sp
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt

keypoint_names = {
    0: "Nose",
    1: "Neck",
    2: "RShoulder",
    3: "RElbow",
    4: "RWrist",
    5: "LShoulder",
    6: "LElbow",
    7: "LWrist",
    8: "MidHip",
    9: "RHip",
    10: "RKnee",
    11: "RAnkle",
    12: "LHip",
    13: "LKnee",
    14: "LAnkle",
    15: "REye",
    16: "LEye",
    17: "REar",
    18: "LEar",
    19: "LBigToe",
    20: "LSmallToe",
    21: "LHeel",
    22: "RBigToe",
    23: "RSmallToe",
    24: "RHeel",
    25: "Background"
}


def smooth_data(input_folder, output_folder, plot_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    if not os.path.exists(plot_folder):
        os.makedirs(plot_folder)
    for index, file_name in enumerate(sorted(os.listdir(input_folder))):
        if file_name.endswith('.npy'):
            keypoint_name = keypoint_names.get(index, "Unknown")
            plot_file_name = f"{keypoint_name}.png"
            plot_path = os.path.join(plot_folder, plot_file_name)

            output_npy_path = os.path.join(output_folder,keypoint_name)
            file_path = os.path.join(input_folder, file_name)
            pose_estimates = np.load(file_path)

            pre_smoothing_x = pose_estimates[:,0].copy()
            pre_smoothing_y = pose_estimates[:,1].copy()

            #Fill in missing X coords
            X_series = pd.Series(pose_estimates[:,0])
            filled_series = X_series.fillna(method='ffill').fillna(0)
            pose_estimates[:,0] = filled_series.to_numpy()

            #fill in missing Y coords
            Y_series = pd.Series(pose_estimates[:,1])
            filled_series = Y_series.fillna(method='ffill').fillna(0)
            pose_estimates[:,1] = filled_series.to_numpy()

            #apply smoothing
            pose_estimates[:,:] = gaussian_filter1d(pose_estimates[:,:], sigma=15, axis=0)
            startingPt = np.nanmedian(pose_estimates[:,0:9], axis=0)
            pose_estimates[:,:] = pose_estimates[:,:] - startingPt

            post_smoothing_x = pose_estimates[:,0]
            post_smoothing_y = pose_estimates[:,1]  # Post-smoothing Y coordinates

            fig = plt.figure(figsize=(10, 7))
            ax = fig.add_subplot(111, projection='3d')  # Create a 3D subplot

            row_indices = np.arange(pose_estimates.shape[0])

            # Plot pre and post smoothing data points in 3D
            ax.scatter(pre_smoothing_x, pre_smoothing_y, row_indices, color='green', label='Pre-Smoothing')
            ax.scatter(post_smoothing_x, post_smoothing_y, row_indices, color='red', label='Post-Smoothing')

            ax.set_title(f'Coordinates Before and After Smoothing for {keypoint_name}')
            ax.set_xlabel('X Coordinate')
            ax.set_ylabel('Y Coordinate')
            ax.set_zlabel('Row Index / Time')
            ax.legend()

            plt.savefig(plot_path)  # Save the 3D plot
            plt.close()  # Close the plot figure
            np.save(output_npy_path, pose_estimates)


#replace the example paths with actual paths
input_folder = output_folder
output_folder = '/content/drive/My Drive/' + input_folder_name + '/outputBody'
plot_folder = '/content/drive/My Drive/' + input_folder_name + '/outputPlots'

smooth_data(input_folder, output_folder, plot_folder)
print('plots printed to ' + plot_folder)

#vectroize each keypoint into a massive matrix


import numpy as np
import os

def transform_keypoints(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.npy'):
            # Load the current keypoints array
            file_path = os.path.join(input_folder, file_name)
            keypoints = np.load(file_path)

            #get rid of confidence score
            keypoints_xy = keypoints[:, :2]


            keypoints_vectorized = keypoints_xy.reshape(-1, 1)


            output_file_path = os.path.join(output_folder, file_name)
            np.save(output_file_path, keypoints_vectorized)

input_folder = output_folder
output_folder = '/content/drive/My Drive/' + input_folder_name + '/vectorized_arrays'
#'/content/gdrive/My Drive/' + input_folder_name + '/vectorized_arrays'

transform_keypoints(input_folder, output_folder)

import numpy as np
import os

def fat_matrix_row_maker(input_folder, big_matrix_file_path):
    # Initialize an empty array for the big matrix. This will hold all stacked vectors.
    fat_matrix_row = None

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.npy'):
            # Load the vectorized keypoints file
            file_path = os.path.join(input_folder, file_name)
            vectorized_keypoints = np.load(file_path, allow_pickle=True)

            # Check if big_matrix is not initialized
            if fat_matrix_row is None:
                fat_matrix_row = vectorized_keypoints
            else:
                # Stack horizontally (add as a new column)
                fat_matrix_row = np.hstack((fat_matrix_row, vectorized_keypoints))

    # Save the big matrix to the specified file path
    file_name="fat_matrix_row"
    output_file_path = os.path.join(output_folder, file_name)
    np.save(output_file_path, fat_matrix_row)

# Specify the input folder containing the vectorized arrays
input_folder = output_folder

#'/content/gdrive/My Drive/<input_folder_name>/vectorized_arrays'

# Specify the path where you want to save the big matrix
output_folder = '/content/drive/My Drive/' + input_folder_name

# Call the function to stack all vectorized keypoints into a big matrix
fat_matrix_row_maker(input_folder, output_folder)

import numpy as np
import os

def update_vertical_matrix(input_folder, output_file_path):
    # Check if the output file already exists
    if os.path.exists(output_file_path):
        # If it exists, load the existing matrix
        vertical_matrix = np.load(output_file_path, allow_pickle=False)
    else:
        # If not, initialize it as None (or an empty array)
        vertical_matrix = None

    # List all .npy files in the input_folder
    npy_files = [f for f in os.listdir(input_folder) if f.endswith('.npy')]

    # Load each .npy file and stack them vertically
    for file_name in npy_files:
        file_path = os.path.join(input_folder, file_name)
        matrix_row = np.load(file_path, allow_pickle=True)



        # If vertical_matrix is None, initialize it with the first matrix_row.
        if vertical_matrix is None:
            vertical_matrix = matrix_row
        else:
            # Stack it vertically
            vertical_matrix = np.vstack((vertical_matrix, matrix_row))

    # Save the updated vertically stacked matrix to the output file path
    np.save(output_file_path, vertical_matrix)
    print(f"Updated matrix saved to {output_file_path}. Shape: {vertical_matrix.shape}")

# Specify the folder containing the fat matrix rows as .npy files
input_folder = output_folder

# Specify the path where you want to save the vertically stacked big matrix
output_file_path = '/content/drive/My Drive/Surfing with Pose Estimation/vertical_stacked_matrix.npy'

# Call the function to stack all fat matrix rows into one big vertically stacked matrix
update_vertical_matrix(input_folder, output_file_path)

Saved NPY file: /content/drive/My Drive/Surfing with Pose Estimation/rhGeneral/example62//output1/62_000000000057_keypoints.npy
Saved NPY file: /content/drive/My Drive/Surfing with Pose Estimation/rhGeneral/example62//output1/62_000000000031_keypoints.npy
Saved NPY file: /content/drive/My Drive/Surfing with Pose Estimation/rhGeneral/example62//output1/62_000000000023_keypoints.npy
Saved NPY file: /content/drive/My Drive/Surfing with Pose Estimation/rhGeneral/example62//output1/62_000000000048_keypoints.npy
Saved NPY file: /content/drive/My Drive/Surfing with Pose Estimation/rhGeneral/example62//output1/62_000000000049_keypoints.npy
Saved NPY file: /content/drive/My Drive/Surfing with Pose Estimation/rhGeneral/example62//output1/62_000000000037_keypoints.npy
Saved NPY file: /content/drive/My Drive/Surfing with Pose Estimation/rhGeneral/example62//output1/62_000000000060_keypoints.npy
Saved NPY file: /content/drive/My Drive/Surfing with Pose Estimation/rhGeneral/example62//output1/62_000

In [ ]:
#add threshhold to mitgate false positive keypoint detection

In [ ]:
# testing bed



import os
import json
import numpy as np
import cv2
import os

#before running script you need the jsons and the original video. place the json folder and the video within a folder in your google drive

# 1. CREATE A FOLDER in google drive that contains the json folder file. also, put the original video in there too.
# 2. DEFINE THESE VARIABLES BEFORE

# parent folder name with a / at the end. for example, if named example, insert 'example/'. dont forget to add the sub-parent folder
input_folder_name = 'Surfing with Pose Estimation/rhGeneral/example25/'

#  folder with jsons and video.
input_jsons = '25_jsons'
input_video = '25.avi'

# input the video name without .avi at the end. for example, if input_video is surfVid.avi, input_jpg will be surfVid
input_jpg = '25'


def json_to_npy_converter(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.endswith(".json"):
            input_file_path = os.path.join(input_folder, file_name)

            with open(input_file_path, 'r') as file:
                try:
                    data = json.load(file)
                    for person in data['people']:
                        keypoints = person['pose_keypoints_2d']
                        keypoints_array = np.array(keypoints).reshape(-1, 3)
                        output_file_path = os.path.join(output_folder, file_name.replace('.json', '.npy'))
                        np.save(output_file_path, keypoints_array)
                        print(f"Saved NPY file: {output_file_path}")
                except Exception as e:
                    print(f"Error processing file {input_file_path}: {e}")

# Replace the example paths with your actual paths
input_folder = '/content/drive/My Drive/' + input_folder_name + input_jsons
output_folder =  '/content/drive/My Drive/' + input_folder_name + '/output1'
json_to_npy_converter(input_folder, output_folder)


# video path extraction

def extract_first_frame(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file.")
        return

    success, frame = cap.read()
    if success:
        output_filename = os.path.splitext(os.path.basename(video_path))[0] + "frame.jpg"
        output_path = os.path.join(os.path.dirname(video_path), output_filename)

        cv2.imwrite(output_path, frame)
        print(f"Saved first frame to {output_path}")
    else:
        print("Failed to extract the first frame.")

    cap.release()
#replace example path with actual path
video_path = '/content/drive/My Drive/' + input_folder_name + input_video
extract_first_frame(video_path)

#finds frame center

from PIL import Image

def get_center_coordinates(image_path):
    with Image.open(image_path) as img:
        with Image.open(image_path):
            height, width = img.size
            center_x = width/2
            center_y = height/2
            return center_x,center_y




image_path = '/content/drive/My Drive/' + input_folder_name + input_jpg + 'frame.jpg'
center_coordinates = get_center_coordinates(image_path)
print(f"{center_coordinates}")

#resize pose estimates to consistent size (height = 220 pixels)

import numpy as np
import os

def rescale_pose_estimates(input_folder, output_folder, target_height=220):


    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.endswith('.npy'):
            file_path = os.path.join(input_folder, file_name)
            pose_estimates = np.load(file_path)

            # Calculate the current height
            max_y = np.max(pose_estimates[:, 1])
            min_y = np.min(pose_estimates[:, 1])
            current_height = max_y - min_y


            scaling_factor = target_height / current_height if current_height > 0 else 0


            pose_estimates[:, :2] *= scaling_factor


            new_file_path = os.path.join(output_folder, file_name)
            np.save(new_file_path, pose_estimates)

#replace the example paths with actual paths
input_folder = output_folder
output_folder = '/content/drive/My Drive/' + input_folder_name + '/output2'


rescale_pose_estimates(input_folder, output_folder)

#center keypoints in frame
#use output from get_center_coordinates for "frame_center" variable (duh)
#input folder is the ouput folder from rescale_pose_estimate

import numpy as np
import os

def center_and_save_pose_estimates(input_folder, output_folder, frame_center):

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)


    for file_name in os.listdir(input_folder):
        if file_name.endswith('.npy'):
            file_path = os.path.join(input_folder, file_name)
            keypoints = np.load(file_path)


            center_of_mass = np.mean(keypoints[:, :2], axis=0)

            difference = center_of_mass-frame_center

            centered_keypoints = keypoints.copy()


            centered_keypoints[:, :2] -= difference


            output_path = os.path.join(output_folder, file_name)
            np.save(output_path, centered_keypoints)


input_folder = output_folder
output_folder = '/content/gdrive/My Drive/' + input_folder_name + '/output3'
frame_center = center_coordinates

center_and_save_pose_estimates(input_folder, output_folder, frame_center)
"subtract difference between reference point and center point from each coordinate"
#check pose heights from each frame

import numpy as np
import os

def calculate_pose_heights(directory):

    heights = {}

    for file_name in os.listdir(directory):
        if file_name.endswith('.npy'):
            file_path = os.path.join(directory, file_name)
            keypoints = np.load(file_path)

            height = np.max(keypoints[:, 1]) - np.min(keypoints[:, 1])
            heights[file_name] = height

    return heights

#replace the example paths with actual paths
directory =  output_folder


pose_heights = calculate_pose_heights(directory)
for file_name, height in pose_heights.items():
    print(f"{file_name}: {height}")

    #seperate pose estimates into arrays corresponding to keypoint ordering ^

import numpy as np
import os


def bodypart_array_maker(input_folder, output_folder):

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    keypoints_accumulator = {} # a dictionary where the key refers to the keypoint index and the value is a list containing the corresponding keypoint accross each frame
    #dictionary contains 25 indeces and each index contains a list that is 319 long
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.npy'):
            file_path = os.path.join(input_folder, file_name)
            keypoints = np.load(file_path)

            for i, keypoint in enumerate(keypoints):
                if i not in keypoints_accumulator:
                    keypoints_accumulator[i] = []   #creates list for each keypoint
                keypoints_accumulator[i].append(keypoint) #populates each list one by one

        for keypoint_index, keypoint_list in keypoints_accumulator.items():
            output_filepath = os.path.join(output_folder, f'keypoint_{keypoint_index}.npy')
            np.save(output_filepath, np.array(keypoint_list))





input_folder = output_folder
output_folder = '/content/drive/My Drive/' + input_folder_name + '/output4'
bodypart_array_maker(input_folder, output_folder)

ValueError: Object arrays cannot be loaded when allow_pickle=False

NameError: name 'fat_matrix_row_file_path' is not defined

Before update:
Matrix file does not exist.

After update:
Matrix successfully loaded. Current shape: (1, 1)


ValueError: Object arrays cannot be loaded when allow_pickle=False